In [8]:
import csv
import json

Code to calculate Crime Gravity and then create a new file with Crime Gravity.

In [9]:
#Upload dates form the JSON files
with open('/content/dict_partecipant_age.json', 'r') as f1_file:
    age_data = json.load(f1_file)

with open('/content/dict_partecipant_status.json', 'r') as f2_file:
    status_data = json.load(f2_file)

with open('/content/dict_partecipant_type.json', 'r') as f3_file:
    type_data = json.load(f3_file)

#Open "police.csv" in reading mode
with open('/content/Police.csv', 'r') as input_file, open('/content/Police_temp.csv', 'w', newline='') as output_file:
    csv_reader = csv.DictReader(input_file)

    #Extract the headers from the input CSV file
    headers = csv_reader.fieldnames
    headers.append('crime_gravity')  #Add new column name

    #Create the writer for the CSV output file
    csv_writer = csv.DictWriter(output_file, fieldnames=headers)
    csv_writer.writeheader()

    for row in csv_reader:
        #Extract values ​​from record fields
        participant_age = row['participant_age_group']
        participant_type = row['participant_type']
        participant_status = row['participant_status']

        #calculate "Crime Gravity" using dates of JSON files
        crime_gravity = (
            age_data.get(participant_age, 0) *
            status_data.get(participant_status, 0) *
            type_data.get(participant_type, 0)
        )

        #Add new values
        row['crime_gravity'] = crime_gravity

        #Write the record in the CSV output file
        csv_writer.writerow(row)


Code to add the missing_ids as foreign key in the Police file for the Custody table.

In [10]:
def aggiungi_missingIDs(file_participant, file_gun, file_geography, file_police, file_output):
    participant_data = {}
    gun_data = {}
    geography_data = {}

    # Leggi il file "participant.csv" e crea un dizionario con le colonne di confronto come chiave
    with open(file_participant, 'r') as participant_file:
        participant_reader = csv.DictReader(participant_file)
        for row in participant_reader:
            key = (row["participant_age_group"], row["participant_status"], row["participant_type"], row["participant_gender"])
            participant_data[key] = row["participant_id"]

    # Leggi il file "gun.csv" e crea un dizionario con le colonne di confronto come chiave
    with open(file_gun, 'r') as gun_file:
        gun_reader = csv.DictReader(gun_file)
        for row in gun_reader:
            key = (row["gun_stolen"], row["gun_type"])
            gun_data[key] = row["gun_id"]

    # Leggi il file "geography.csv" e crea un dizionario con le colonne di confronto come chiave
    with open(file_geography, 'r') as geography_file:
        geography_reader = csv.DictReader(geography_file)
        for row in geography_reader:
            key = (row["latitude"], row["longitude"])
            geography_data[key] = row["geo_id"]

    # Apri il file "police.csv" in modalità lettura
    with open(file_police, 'r') as police_file:
        police_reader = csv.DictReader(police_file)

        # Prendi l'intestazione dal file "police.csv" e aggiungi le nuove colonne
        header = police_reader.fieldnames
        header.extend(["participant_id", "gun_id", "geo_id"])

        # Apri il file di output in modalità scrittura
        with open(file_output, 'w', newline='') as output_file:
            csv_writer = csv.DictWriter(output_file, fieldnames=header)
            csv_writer.writeheader()

            for row in police_reader:
                participant_key = (row["participant_age_group"], row["participant_status"], row["participant_type"], row["participant_gender"])
                gun_key = (row["gun_stolen"], row["gun_type"])
                geography_key = (row["latitude"], row["longitude"])

                # Aggiungi i valori delle colonne "participant_id", "gun_id", e "geography_id" dai rispettivi dizionari
                row["participant_id"] = participant_data.get(participant_key, "")
                row["gun_id"] = gun_data.get(gun_key, "")
                row["geo_id"] = geography_data.get(geography_key, "")

                csv_writer.writerow(row)

file_participant = "/content/Participant.csv"
file_gun = "/content/Gun.csv"
file_geography = "/content/Geography.csv"
file_police = "/content/Police_temp.csv"
file_output = "/content/Police_final.csv"
aggiungi_missingIDs(file_participant, file_gun, file_geography, file_police, file_output)